# Timezone - Get Timezone By Coordinates

https://azure.microsoft.com/en-us/products/azure-maps/?msockid=2e39c66c693c66a5151fd200687567d0

https://learn.microsoft.com/en-us/azure/azure-maps/

In [1]:
import json
import os
import requests
import sys

from datetime import datetime
from dotenv import load_dotenv
from typing import Optional, Dict, Any

In [2]:
load_dotenv("azure.env")

True

In [3]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

## Helper

In [4]:
class AzureMapsTimezone:
    """
    Azure Maps Timezone API client for getting timezone information by coordinates.
    
    API Reference: https://docs.microsoft.com/en-us/rest/api/maps/timezone/gettimezonebycoordinates
    """
    
    def __init__(self, subscription_key: str):
        """
        Initialize the Azure Maps Timezone client.
        
        Args:
            subscription_key (str): Your Azure Maps subscription key
        """
        self.subscription_key = subscription_key
        self.base_url = "https://atlas.microsoft.com/timezone/byCoordinates/json"
        
    def get_timezone_by_coordinates(
        self, 
        latitude: float, 
        longitude: float,
        timestamp: Optional[str] = None,
        accept_datetime: Optional[str] = None,
        api_version: str = "1.0"
    ) -> Dict[str, Any]:
        """
        Get timezone information for given coordinates.
        
        Args:
            latitude (float): Latitude coordinate (-90 to 90)
            longitude (float): Longitude coordinate (-180 to 180)
            timestamp (str, optional): Reference time in ISO 8601 format (e.g., "2023-01-15T12:00:00Z")
            accept_datetime (str, optional): Alternative timestamp format
            api_version (str): API version (default: "1.0")
            
        Returns:
            Dict[str, Any]: Timezone information including ID, offset, and names
            
        Raises:
            requests.RequestException: If the API request fails
            ValueError: If coordinates are invalid
        """
        # Validate coordinates
        if not -90 <= latitude <= 90:
            raise ValueError("Latitude must be between -90 and 90 degrees")
        if not -180 <= longitude <= 180:
            raise ValueError("Longitude must be between -180 and 180 degrees")
            
        # Prepare query parameters
        params = {
            "api-version": api_version,
            "subscription-key": self.subscription_key,
            "query": f"{latitude},{longitude}"
        }
        
        # Add optional parameters
        if timestamp:
            params["timeStamp"] = timestamp
        if accept_datetime:
            params["Accept-Datetime"] = accept_datetime
            
        # Make the API request
        try:
            response = requests.get(self.base_url, params=params, timeout=30)
            response.raise_for_status()  # Raises HTTPError for bad responses
            
            return response.json()
            
        except requests.RequestException as e:
            raise requests.RequestException(f"Failed to get timezone data: {str(e)}")
    
    def get_current_timezone(self, latitude: float, longitude: float) -> Dict[str, Any]:
        """
        Get current timezone information for given coordinates.
        
        Args:
            latitude (float): Latitude coordinate
            longitude (float): Longitude coordinate
            
        Returns:
            Dict[str, Any]: Current timezone information
        """
        current_timestamp = datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")
        return self.get_timezone_by_coordinates(latitude, longitude, timestamp=current_timestamp)


## Examples

In [5]:
timezone_client = AzureMapsTimezone(os.getenv('AZURE_MAPS_KEY'))

In [6]:
latitude = 48.85341
longitude = 2.3488

print(f"Getting timezone for coordinates: {latitude}, {longitude}")
        
# Get current timezone information
timezone_info = timezone_client.get_current_timezone(latitude, longitude)
        
# Display results
print("\n=== Timezone Information ===")
print(json.dumps(timezone_info, indent=2))

Getting timezone for coordinates: 48.85341, 2.3488

=== Timezone Information ===
{
  "Version": "2025b",
  "ReferenceUtcTimestamp": "2025-09-01T08:03:31Z",
  "TimeZones": [
    {
      "Id": "Europe/Paris",
      "Names": {
        "ISO6391LanguageCode": "en",
        "Generic": "Central European Time",
        "Standard": "Central European Standard Time",
        "Daylight": "Central European Summer Time"
      },
      "ReferenceTime": {
        "Tag": "CEST",
        "StandardOffset": "01:00:00",
        "DaylightSavings": "01:00:00",
        "WallTime": "2025-09-01T10:03:31+02:00",
        "PosixTzValidYear": 2025,
        "PosixTz": "CET-1CEST,M3.5.0,M10.4.0/3",
        "Sunrise": "2025-09-01T07:07:07.8811458+02:00",
        "Sunset": "2025-09-01T20:34:19.8840349+02:00"
      }
    }
  ]
}


In [7]:
# Extract specific information
if "TimeZones" in timezone_info and timezone_info["TimeZones"]:
    tz = timezone_info["TimeZones"][0]
    print(f"Timezone ID: {tz.get('Id', 'N/A')}")
    print(f"Standard Name: {tz.get('Names', {}).get('Standard', 'N/A')}")
    print(f"Daylight Name: {tz.get('Names', {}).get('Daylight', 'N/A')}")
    print(f"UTC Offset: {tz.get('ReferenceUtcTimestamp', 'N/A')}")

Timezone ID: Europe/Paris
Standard Name: Central European Standard Time
Daylight Name: Central European Summer Time
UTC Offset: N/A


In [8]:
# Example with specific timestamp
print("Getting timezone for specific timestamp...\n")
        
specific_timestamp = "2023-07-15T12:00:00Z"
timezone_info_specific = timezone_client.get_timezone_by_coordinates(latitude, longitude, timestamp=specific_timestamp)
        
print(f"Timezone info for {specific_timestamp}:")
print(json.dumps(timezone_info_specific, indent=2))

Getting timezone for specific timestamp...

Timezone info for 2023-07-15T12:00:00Z:
{
  "Version": "2025b",
  "ReferenceUtcTimestamp": "2023-07-15T12:00:00Z",
  "TimeZones": [
    {
      "Id": "Europe/Paris",
      "Names": {
        "ISO6391LanguageCode": "en",
        "Generic": "Central European Time",
        "Standard": "Central European Standard Time",
        "Daylight": "Central European Summer Time"
      },
      "ReferenceTime": {
        "Tag": "CEST",
        "StandardOffset": "01:00:00",
        "DaylightSavings": "01:00:00",
        "WallTime": "2023-07-15T14:00:00+02:00",
        "PosixTzValidYear": 2023,
        "PosixTz": "CET-1CEST,M3.4.0,M10.5.0/3",
        "Sunrise": "2023-07-15T06:02:32.9709572+02:00",
        "Sunset": "2023-07-15T21:50:30.0340793+02:00"
      }
    }
  ]
}
